# Time Series Exploratory Notebook

In [ ]:
# load dependencies
import matplotlib.pyplot as plt
import statsmodels
import pandas as pd
import numpy as np

## Historic Data

### Import data

In [ ]:
# historic dataset
path = "../data/raw/"
file = "100014.csv"
df = pd.read_csv(path + file, sep=";")

### Dataset info

In [ ]:
df.info()

**Description:** There are 11 columns with information in the dataset. Most of them are of dtype object. Two columns `Anzahl frei` and `Total Plätze` are of type int64 which is a good sign since we only expect nummeric characters there.

### Columns with na values

In [ ]:
df.isna().any()

**Description:** There are no missing values in any columns of the historic dataset. All entries seem to be complete.

### Nr of garages in the dataset

In [ ]:
df[["id", "id2", "title"]].drop_duplicates().reset_index(drop=True)

**Description:** There are a total of 16 distinct garages in the dataset. 

### Nr of entries per garage

In [ ]:
df[["title", "published"]].groupby("title").count()

**Description:** There are only two garage: "Parkhaus Kunstmuseum" and "Parkhaus City" which contain less entries than the rest.

### Nr unique of `Total Parkplätze`

In [ ]:
df[["title", "total"]].groupby("title").nunique()

**Description:** Most garages have a constant number of parking spots for the duration of the dataset. However there are also two garages with 2 different totals: "Parkhaus Bad. Bahnhof" and "Parkhaus Claramatte". In a next step, we want to investigate the reason for this.

In [ ]:
def show_total_spots_timeline(df, name, last_n_entries=None):
    '''
    Creates a lineplot with the total nr of available parking spots for as single garage
    '''
    df = df.copy()
    df = df[df["title"]==name][["published", "total"]].sort_values(by = "published").reset_index(drop=True)
    
    df["published"] = pd.to_datetime(df["published"])
    
    if last_n_entries:
        df = df[-last_n_entries:]
        
    df = df.sort_values(by="published")
    
    plt.figure(figsize=(16,5))
    plt.title(f"Total nr of parking spots for: {name}")
    plt.plot(df["published"], df["total"])
    plt.xlabel("Timeline")
    plt.ylabel("Total spots")
    plt.show()

### Timeline for "Parkhaus Bad. Bahnhof"

In [ ]:
show_total_spots_timeline(df, name = "Parkhaus Bad. Bahnhof")    
show_total_spots_timeline(df, name = "Parkhaus Bad. Bahnhof", last_n_entries=7*24)

### Timeline for "Parkhaus Claramatte"

In [ ]:
show_total_spots_timeline(df, name = "Parkhaus Claramatte")    
show_total_spots_timeline(df, name = "Parkhaus Claramatte", last_n_entries=7*24)

The parking garages can sometimes change the number of public parking spaces. The exact reason is unknown. The following link has more information about it.

https://data.bs.ch/explore/dataset/100014/comments/?disjunctive.title&sort=published&dataChart=eyJxdWVyaWVzIjpbeyJjaGFydHMiOlt7InR5cGUiOiJsaW5lIiwiZnVuYyI6Ik1JTiIsInlBeGlzIjoiZnJlZSIsInNjaWVudGlmaWNEaXNwbGF5Ijp0cnVlLCJjb2xvciI6InJhbmdlLUFjY2VudCJ9XSwieEF4aXMiOiJwdWJsaXNoZWQiLCJtYXhwb2ludHMiOiIiLCJ0aW1lc2NhbGUiOiJob3VyIiwic29ydCI6IiIsInNlcmllc0JyZWFrZG93biI6InRpdGxlIiwiY29uZmlnIjp7ImRhdGFzZXQiOiIxMDAwMTQiLCJvcHRpb25zIjp7ImRpc2p1bmN0aXZlLnRpdGxlIjp0cnVlLCJzb3J0IjoicHVibGlzaGVkIn19fV0sImRpc3BsYXlMZWdlbmQiOnRydWUsImFsaWduTW9udGgiOnRydWV9 



### delete not used columns

In [ ]:
df

In [ ]:
df_modified = df[['published', 'free', 'id2', 'total', 'geo_point_2d']].copy()

### Datatypes

In [ ]:
df_modified.published = pd.to_datetime(df_modified.published)
print('date check')
# df_modified['id2'] = df_modified['id2'].apply(lambda x: str(x))
df_modified['id2'] = df_modified['id2'].astype(str)
print('string check')
df_modified['geo_point_2d'] = df_modified['geo_point_2d'].astype(str)
df_modified['latitude'] = df_modified.geo_point_2d.apply(lambda x: x.strip().split(',')[0])
df_modified['longitude'] = df_modified.geo_point_2d.apply(lambda x: x.strip().split(',')[1])

df_modified['latitude'] = df_modified.latitude.astype(float)
df_modified['longitude'] = df_modified.longitude.astype(float)
df_modified.drop('geo_point_2d', axis=1, inplace=True)

In [ ]:
df_modified.info()
df_modified

### repeated lines

In [ ]:
df_modified.duplicated().sum()

There are no duplicated rows

### checking data with logical conditions 

**All dates should be consecutive**

In [ ]:
date_number = df_modified.groupby('published').size()
date_number

In [ ]:
date_number

In [ ]:
all = date_number.count()
a16 = (date_number==16).sum()
a15 = (date_number==15).sum()
print((a16 + a15) == all)

For every date we have 15 or 16 entries. This means that for every date are enough entries.

In [ ]:
leng = len([i for i in date_number.index])
plt.scatter(y=range(leng), x=[date_number.index])

The dates are consecutive

In [ ]:
df_modified.groupby(['id2', 'total']).size()

Two garages don't have data for the whole time (city and kunstmuseum). The other have data for the whole period. The total number of places stays the same except the two garages we analysed above.

**Number of free parking spots should always be less or equal to the total number**

In [ ]:
df_modified[df_modified['free'] > df_modified['total']]

There are rows that have more free parking spots than total parking spots.

Analysis of the lines with too much free parking spots

In [ ]:
df_too_much_free = df_modified[df_modified['free'] > df_modified['total']]
df_too_much_free.groupby('id2').size().sort_values(ascending=False).plot(kind='bar', figsize=(16,5))
plt.title('how many times a parking garage had more free spots than total available')
plt.ylabel('number of lines with more free spots than total available')
plt.xlabel('parking garage')
plt.show()

In [ ]:
def list_too_much_free(parking):
    '''
    returns a list with lists. The inner lists contain a series of indexer for all the times a parking garage had more free spots than total available
    plus the one before and after
    '''
    df_parking = df_modified[df_modified['id2'] == parking].reset_index(drop=True)
    list_with_periods = []
    i = 0
    while i < df_parking.shape[0]:
        if i != 0:
            if df_parking.loc[i, 'free'] > df_parking.loc[i, 'total']:
                list_ = []
                while df_parking.loc[i, 'free'] > df_parking.loc[i, 'total']:
                    if df_parking.loc[i, 'free'] > df_parking.loc[i, 'total']:
                        if df_parking.loc[i-1, 'free'] > df_parking.loc[i-1, 'total']:
                            list_.append(i)
                        elif df_parking.loc[i+1, 'free'] > df_parking.loc[i+1, 'total']:
                            list_.append(i)
                        elif df_parking.loc[i+1, 'free'] < df_parking.loc[i+1, 'total']:
                            list_.append(i)
                            list_.append(i+1)
                        else:
                            list_.append(i-1)
                            list_.append(i)
                    i += 1
                list_with_periods.append(list_)
            else:
                i += 1
        else: 
            list_ = []
            if df_parking.loc[i, 'free'] > df_parking.loc[i, 'total']:
                while df_parking.loc[i, 'free'] > df_parking.loc[i, 'total']:
                    if df_parking.loc[i, 'free'] > df_parking.loc[i, 'total']:
                        if df_parking.loc[i+1, 'free'] > df_parking.loc[i+1, 'total']:
                            list_.append(i)
                        elif df_parking.loc[i+1, 'free'] < df_parking.loc[i+1, 'total']:
                            list_.append(i)
                            list_.append(i+1)
                    i += 1
                list_with_periods.append(list_)
            else:
                i += 1
    return list_with_periods

In [ ]:
postbasel_free = list_too_much_free('postbasel')
df_postbasel = df_modified[df_modified['id2'] == 'postbasel'].reset_index(drop=True)

In [ ]:
list_diff = [np.absolute(df_postbasel.loc[i[0], 'free'] - df_postbasel.loc[i[-1], 'free']) for i in postbasel_free]
plt.hist(100*(list_diff/df_postbasel.loc[0,'total']), bins=30)
plt.title('difference between the value before the value with too much spots and the value after the value with too much spots for postbasel')
plt.ylabel('count')
plt.xlabel('change in %')
plt.show()
print(np.mean(list_diff))
print(np.std(list_diff))

In [ ]:
list_centralbahnparking = list_too_much_free('centralbahnparking')
df_centralbahnparking = df_modified[df_modified['id2'] == 'centralbahnparking'].reset_index(drop=True)

In [ ]:
list_diff = [np.absolute(df_centralbahnparking.loc[i[0], 'free'] - df_centralbahnparking.loc[i[-1], 'free']) for i in list_centralbahnparking]
plt.boxplot(100*(list_diff/df_centralbahnparking.loc[0,'total']))
plt.title('difference between the value before the value with too much spots and the value after the value with too much spots for central bahn parking')
plt.ylabel('difference')
plt.show()
print(np.mean(list_diff))
print(np.std(list_diff))

In [ ]:
list_badbahnhof = list_too_much_free('badbahnhof')
df_badbahnhof = df_modified[df_modified['id2'] == 'badbahnhof'].reset_index(drop=True)

In [ ]:
list_diff = [np.absolute(df_badbahnhof.loc[i[0], 'free'] - df_badbahnhof.loc[i[-1], 'free']) for i in list_badbahnhof]
plt.boxplot(100*(list_diff/df_badbahnhof.loc[0,'total']))
plt.title('difference between the value before the value with too much spots and the value after the value with too much spots for badischer bahnhof')
plt.ylabel('difference')
plt.show()
print(np.mean(list_diff))
print(np.std(list_diff))

In [ ]:
list_clarahuus = list_too_much_free('clarahuus')
df_clarahuus = df_modified[df_modified['id2'] == 'clarahuus'].reset_index(drop=True)

In [ ]:
list_diff = [np.absolute(df_clarahuus.loc[i[0], 'free'] - df_clarahuus.loc[i[-1], 'free']) for i in list_clarahuus]
plt.boxplot(list_diff)
plt.title('difference between the value before the value with too much spots and the value after the value with too much spots for clarahuus')
plt.ylabel('difference')
plt.show()
print(np.mean(list_diff))
print(np.std(list_diff))

In [ ]:
list_messe = list_too_much_free('messe')
df_messe = df_modified[df_modified['id2'] == 'messe'].reset_index(drop=True)

In [ ]:
list_diff = [np.absolute(df_messe.loc[i[0], 'free'] - df_messe.loc[i[-1], 'free']) for i in list_messe]
plt.boxplot(list_diff)
plt.title('difference between the value before the value with too much spots and the value after the value with too much spots for messe')
plt.ylabel('difference')
plt.show()
print(np.mean(list_diff))
print(np.std(list_diff))

In [ ]:
list_bahnhofsued = list_too_much_free('bahnhofsued')
df_bahnhofsued = df_modified[df_modified['id2'] == 'bahnhofsued'].reset_index(drop=True)

In [ ]:
list_diff = [np.absolute(df_bahnhofsued.loc[i[0], 'free'] - df_bahnhofsued.loc[i[-1], 'free']) for i in list_bahnhofsued]
plt.boxplot(list_diff)
plt.title('difference between the value before the value with too much spots and the value after the value with too much spots for bahnhof süd')
plt.ylabel('difference')
plt.show()
print(np.mean(list_diff))
print(np.std(list_diff))

Claramatte is deleted because almost 1/3 of the data is wrong.

In the other parkings, the free spots (where free parking spots > total spots) are set to Nan's. Then this nans are interpolated. The interpolation is done with the method "linear". This means that the values are interpolated with the values of the previous and next entry. This is done for every garage separately. If several Nan's are in a row, the interpolation is done with the values of the previous and next entry that are not Nan's (also linear).

In [ ]:
# replace all free parkings with nan if free is bigger than total
def replace_with_nan(df):
    i = 0
    while i < len(df):
        if df.loc[i, 'free'] > df.loc[i, 'total']:
            df.loc[i, 'free'] = np.nan
            i += 1
        else:
            i += 1
    return df
df_cleaned = replace_with_nan(df_modified)

In [ ]:
df_cleaned[(df_cleaned['id2'] == 'postbasel') & df_cleaned['free'].isna()]
df_cleaned[(df_cleaned['id2'] == 'postbasel')].loc[6458:6465,:]

In [ ]:
df_cleaned[(df_cleaned['id2'] == 'postbasel')].interpolate(method='linear').loc[6458:6465,:].round()

In [ ]:
# # create a timeseries dataframe
# df_timeseries = df_cleaned[['id2', 'free', 'published']]
# df_timeseries = df_timeseries.pivot(index='published', columns='id2', values='free')
# df_timeseries

In [ ]:
# # df with information about the parkings
# df_parking = df_cleaned.groupby(['id2']).agg({'total': ['min', 'max'], 'latitude': ['max'], 'longitude': ['max']})
# df_parking

In [ ]:
# # create a timeseries dataframe

for i in df_cleaned['id2'].unique():
    df_cleaned.loc[df_cleaned['id2'] == i,'free'] = df_cleaned.loc[df_cleaned['id2'] == i,'free'].interpolate(method='linear')
# df_cleaned = df_cleaned.iloc[:-2,:]
df_cleaned[df_cleaned['free'].isna()]
# df_cleaned['free'] = df_cleaned['free'].round().astype(int)

In [ ]:
all_lines = 0
for i in df['free'].unique():
    print(i, df[df['id2']==i].shape)
    all_lines += i[1].shape[0]
print(all_lines)

The final dataframe we get, contains all the data for all the parkings

**all parkings should have the same coordinates**

In [ ]:
df_modified.groupby(['id2', 'lattitude']).size()

For every garage is only one coordinate.

### Timeseries for every garage

In [ ]:
# n_weeks is approximately the number of weeks to be shown
n_weeks=1
garages = df_modified.id2.unique()
for i in garages:
    g1 = df[df['id2'] == i]
    plt.plot(g1.published[-188*n_weeks:], g1.free[-188*n_weeks:])
    print('check')
    plt.title('Timeseries for ' + i)
    plt.xlabel('date')
    plt.ylabel('Free parking spots')
    plt.show()